In [40]:
import pandas as pd

In [41]:
df = pd.read_csv('nyc-rolling-sales.csv')
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84548 entries, 0 to 84547
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Unnamed: 0                      84548 non-null  int64 
 1   BOROUGH                         84548 non-null  int64 
 2   NEIGHBORHOOD                    84548 non-null  object
 3   BUILDING CLASS CATEGORY         84548 non-null  object
 4   TAX CLASS AT PRESENT            84548 non-null  object
 5   BLOCK                           84548 non-null  int64 
 6   LOT                             84548 non-null  int64 
 7   EASE-MENT                       84548 non-null  object
 8   BUILDING CLASS AT PRESENT       84548 non-null  object
 9   ADDRESS                         84548 non-null  object
 10  APARTMENT NUMBER                84548 non-null  object
 11  ZIP CODE                        84548 non-null  int64 
 12  RESIDENTIAL UNITS               84548 non-null

In [42]:
df = df[df['SALE PRICE'] != 0]
df.shape

(84548, 22)

In [43]:
null_sale_price_count = df['SALE PRICE'].isnull().sum()
print(f"Number of null values in SALE PRICE: {null_sale_price_count}")

df = df.dropna(subset=['SALE PRICE'])
df.shape

Number of null values in SALE PRICE: 0


(84548, 22)

In [46]:
df = pd.read_csv('nyc-rolling-sales_cleaned.csv')
df.columns


Index(['Unnamed: 0', 'NEIGHBORHOOD', 'TAX CLASS AT PRESENT', 'BLOCK', 'LOT',
       'EASE-MENT', 'BUILDING CLASS AT PRESENT', 'ADDRESS', 'APARTMENT NUMBER',
       'ZIP CODE', 'RESIDENTIAL UNITS', 'COMMERCIAL UNITS', 'TOTAL UNITS',
       'LAND SQUARE FEET', 'GROSS SQUARE FEET', 'YEAR BUILT',
       'BUILDING CLASS AT TIME OF SALE', 'SALE PRICE', 'SALE DATE',
       'BOROUGH_brooklyn', 'BOROUGH_manhattan', 'BOROUGH_queens',
       'BOROUGH_staten island',
       'BUILDING CLASS CATEGORY_02 TWO FAMILY DWELLINGS                    ',
       'BUILDING CLASS CATEGORY_03 THREE FAMILY DWELLINGS                  ',
       'BUILDING CLASS CATEGORY_04 TAX CLASS 1 CONDOS                      ',
       'BUILDING CLASS CATEGORY_05 TAX CLASS 1 VACANT LAND                 ',
       'BUILDING CLASS CATEGORY_06 TAX CLASS 1 - OTHER                     ',
       'BUILDING CLASS CATEGORY_07 RENTALS - WALKUP APARTMENTS             ',
       'BUILDING CLASS CATEGORY_08 RENTALS - ELEVATOR APARTMENTS        